# Lab 1.2 A simple lab assistant

In [1]:
from igfold import IgFoldRunner

/opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
sequences = {
    "L": "D"
}
pred_pdb = "output2/my_antibody.pdb"

igfold = IgFoldRunner()
result = igfold.fold(
    pred_pdb, # Output PDB file
    sequences=sequences, # Antibody sequences
    do_refine=False, # Refine the antibody structure with PyRosetta
    do_renum=False, # Renumber predicted antibody structure (Chothia)
)

/opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/lightning_lite/utilities/cloud_io.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retur


    The code, data, and weights for this work are made available for non-commercial use 
    (including at commercial entities) under the terms of the JHU Academic Software License 
    Agreement. For commercial inquiries, please contact awichma2[at]jhu.edu.
    License: https://github.com/Graylab/IgFold/blob/main/LICENSE.md
    
Loading 4 IgFold models...
Using device: cpu
Loading /opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_1.ckpt...
Loading /opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_2.ckpt...
Loading /opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_3.ckpt...
Loading /opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_5.ckpt...
Successfully loaded 4 IgFold models.
Loaded AntiBERTy model.


/opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/igfold/utils/coordinates.py:18: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Cross.cpp:66.)
  n_vec = (b_coord - a_coord).expand(bc_vec.shape).cross(bc_vec)
/opt/homebrew/Caskroom/miniconda/base/envs/workshop/lib/python3.12/site-packages/igfold/model/components/IPABlock.py:160: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with disable_tf32(), autocast(enabled = False):


Completed folding in 0.39 seconds.


In [17]:
result.coords

tensor([[[[-2.8587, -0.3114,  2.1917],
          [-2.6634, -2.3373,  1.3355],
          [-1.6790, -3.9292,  2.8000],
          [-1.5698, -2.8136, -0.8168],
          [-1.5139, -5.0957,  2.4433]]]])

In [12]:
import py3Dmol
def load_protein(pdb_file_path, width=800, height=600):

    """
    Load a protein structure from a PDB file and display it using py3Dmol.
    pdb_file_path: str, path to the PDB file
    width: int, width of the viewer in pixels
    height: int, height of the viewer in pixels
    return: py3Dmol.view object
    """
    
    with open(pdb_file_path) as ifile:
        pdb_data = "".join([x for x in ifile])
    
    view = py3Dmol.view(width=width, height=height)
    view.addModelsAsFrames(pdb_data)
    
    for line in pdb_data.split("\n"):
        split = line.split()
        if len(split) == 0 or split[0] != "ATOM":
            continue
        # Assuming the B-factor is at position 10 (you may need to adjust this based on your PDB format)
        b_factor = float(split[10])
        if b_factor > 90:
            color = "blue"
        elif 70 <= b_factor <= 90:
            color = "cyan"
        elif 50 <= b_factor < 70:
            color = "yellow"
        else:
            color = "orange"
        
        # Atom serial numbers typically start from 1, hence idx should be used directly
        idx = int(split[1])
        
        # Style should be set per atom id
        view.setStyle({'model': -1, 'serial': idx}, {"cartoon": {'color': color}})
    view.zoomTo()
    return view

In [13]:
load_protein("../research_assistant/output/esmfold_result/protein_1.pdb")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.